In [9]:
%pip install pytubefix

### Youtube에서 오디오 추출하기

In [7]:
import os
from pytubefix import YouTube

def downloadYouTube(videourl, path):
    yt = YouTube(videourl)
    yt = yt.streams.filter(only_audio=True).first()
    if not os.path.exists(path):
        os.makedirs(path)
    return yt.download(path)

url = "https://www.youtube.com/watch?v=isUudT58Xfk"
audio_filename = downloadYouTube(url, "audio")

### 로컬에서 파일 가져오기

In [2]:
audio_filename = "../../Assets/audio/2024-08-21 10-33-51.mp4"

### colab에서 파일 가져오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

audio_filename = '/content/drive/MyDrive/OH_MY_GIRL_CLOSER.mp4'

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

### OpenAI API 이용하기

입력할 수 있는 최대 파일 크기는 25 MB이며 입력 파일의 형식은 mp3, mp4, mpeg, mpga, m4a, wav, webm 형식을 지원합니다. 파일 크기가 크다면 pydub 등을 이용해 잘라서 사용하면 됩니다.

response_format은 text외에도 `json, verbose_json, srt, vtt` 형식을 사용할 수 있습니다.

In [ ]:
import os
from openai import OpenAI 

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

with open(audio_filename, "rb") as audio_file:
    transcript = client.audio.transcripitons.create(
        file = audio_file,
        model = "whisper-1",
        response_format="text",
        language="ko",
    )
print(transcript)

### Whisper Open Source 이용하기

#### ffmpeg 설치하기

\# on Ubuntu or Debian

`sudo apt update && sudo apt install ffmpeg`

\# on MacOS using Homebrew

`brew install ffmpeg`

In [5]:
%pip install git+https://github.com/openai/whisper.git -q
%pip install torch tiktoken

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import whisper

model = whisper.load_model("medium")
result = model.transcribe(audio_filename)
transcript = result["text"]
print(transcript)

/opt/anaconda3/envs/llm/lib/python3.12/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 잘 보입니다 안녕하세요 안녕하세요 김정은이 형 지금 내 머리 보고 웃는 거 아니야? 아니에요 베트남 헤어스타일인가 보네요 요즘 아무 생각 없이 신경 안 써서 베트남 미용실 가서 머리 깎아줘 눈 감고 눈 떴더니 이렇게 돼 있더라고 참아라 보기 싫어도 어쩔 수 없다 제가 이 세면이랑 내가 얘기할 내용은 매일 업무 지연 내역이랑 소수 관리 내역이랑 업데이트 진행 내역이 세 가지가 주제인데 매일 업무 지연 내역에서 지금 진행하는 내용들을 내가 어느 정도 모니터링을 해서 보고는 있는데 이게 파악된 내용들을 이제 앞으로 케이스 형태로 작성해도 되고 아니면 문서 형태로 작성해도 되고 히스터리로 계속 남기면서 그 내용들을 확인을 하고 김정은 부장 그 내용들을 보면서 잘못된 부분들에 있으면 코멘트를 하고 정리를 하고 업데이트를 하는 방식으로 이렇게 진행해야 될 것 같다는 생각은 좀 들어 어떻게 해야 될지는 지금 여기 실무자 분들이 그 부분을 잘 정리해 가지고 가장 좋은 방안을 만들어야 될 거라고 생각을 하고 있고 그리고 9월 달부터는 이제 문형 메인 터너스 정도 수준에서는 일을 한 건씩이라도 한 번씩 테스트를 하면서 업무 지연을 해보고 그 외에 서비스 지원권이나 내용들을 보니까 도대체 이걸 이렇게까지 논의를 해야 될 내용들이냐 라는 것들도 꽤 많이 보이는데 그런 것들은 또 어떤 식으로 업무 지연을 해야 되는지 아니면 누가 업무를 처리해야 되는지 이런 것들도 좀 분류를 해야 되지 않겠나 이런 생각이 좀 들어 기본적으로 메일 대화방을 보니까 어제 건으로 예를 들어서 메일을 발송하는 어떤 프로그램의 우리를 우리한테 얘기를 하고 그거를 지원하는 자법적으로는 확실히 대처를 안 하니까 거기 대화방에 메세지만 한 30건이 왔다 갔다 하더라고 그걸로 그거 실제적으로 피로도가 높아지지 않아? 그런 내용들 있으면? 다른 프로그램이랑 겹치면 그런 일이 발생하는데요 아웃룩이나 이런 거는 냅두더라도 다른 회사에서 만든 그룹에가 있고 우리가 만든 그룹에가 있고 그렇게 되면 이 회사와 우리 회사 간 

### 요약하기

In [5]:
from string import Template

summary_prompt = Template("""
아래 내용은 사용자의 음성을 텍스트로 변환한 내용입니다.

내용을 요약해 주세요.

내용: $input
요약:
""")

In [6]:
import os
from openai import OpenAI 

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def summary_voice(transcript):
  chat_completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[{"role": "user", "content": summary_prompt.substitute(input=transcript)}],
  )

  return chat_completion.choices[0].message.content

summary_voice(transcript)

'사용자는 머리 스타일에 대한 농담으로 시작해, 업무 관련 내용을 중심으로 세 가지 주제를 다루었습니다. 첫째, 매일 업무 지연 내역 및 소수 관리 내역을 기록하고, 업무 상황을 모니터링하며 문제점을 찾아 코멘트 및 업데이트를 해야 한다고 강조했습니다. 둘째, 업데이트 시 고객에게 공지하는 절차의 중요성을 언급하며, 사전 준비와 정보 제공이 필요하다고 설명했습니다. 마지막으로, 새로운 기술적 이슈와 관련하여 테스트 및 검토가 중요하다고 하며, 팀원들과 소통하여 문제를 해결해 나가야 한다고 강조했습니다. 이 과정에서 문서화와 정보 공유의 필요성도 강조되었습니다.'